In [ ]:


import os
import mediapipe as mp
import cv2
import numpy as np
import sys
sys.path.append('/projects/sign_language')
from utils.utils import createFolder
# sys.path.append('/Users/ganzorig/Desktop/Codes/sign_language')


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
# For webicam input:
#path = '/Users/ganzorig/Desktop/letters_splitted/0/test/aeen/Munkhuush_aeen.mp4'
data_path = '/projects/ZHO/final/videos/'
dest_path = '/projects/ZHO/formats/skeleton_sign/'
createFolder(dest_path)

qqqq = 'Munkhuush/Education/science.mp4'
people = os.listdir(data_path)

In [ ]:
for person in people:
    print('person:',person)
    categories = os.listdir(os.path.join(data_path, person))
    for category in categories:
        videos = os.listdir(os.path.join(data_path, person, category))
        createFolder(os.path.join(dest_path, person, category), True)
        print('category:',category)
        for video in videos:

            cap = cv2.VideoCapture(data_path)

            frame_width, frame_height = int(cap.get(3)), int(cap.get(4))
            #(1920, 1080)
            fps = cap.get(cv2.CAP_PROP_FPS)
            out = cv2.VideoWriter(os.path.join(dest_path, person, category, video), cv2.VideoWriter_fourcc(
                *'MP4V'), fps, (frame_width, frame_height))

            with mp_holistic.Holistic(
                    min_detection_confidence=0.2,
                    min_tracking_confidence=0.2) as holistic:
                while cap.isOpened():
                    success, image = cap.read()
                    if not success:
                        print("Ignoring empty camera frame.")
                        # If loading a video, use 'break' instead of 'continue'.
                        break

                    # To improve performance, optionally mark the image as not writeable to
                    # pass by reference.
                    image.flags.writeable = False

                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    shape = image.shape
                    # print(shape)
                    blank = np.zeros_like(image)
                    results = holistic.process(image)
                    print('ff:', results.right_hand_landmarks)
                    # Draw landmark annotation on the image.
                    image.flags.writeable = True
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                    # Right Hand
                    mp_drawing.draw_landmarks(
                        blank, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

                    # Left Hand
                    mp_drawing.draw_landmarks(
                        blank, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

                    mp_drawing.draw_landmarks(
                        blank,
                        results.face_landmarks,
                        mp_holistic.FACEMESH_CONTOURS,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing_styles
                        .get_default_face_mesh_contours_style())
                    mp_drawing.draw_landmarks(
                        blank,
                        results.pose_landmarks,
                        mp_holistic.POSE_CONNECTIONS,
                        landmark_drawing_spec=mp_drawing_styles
                        .get_default_pose_landmarks_style())
                    # Flip the image horizontally for a selfie-view display.
                    out.write(blank)
                    #cv2.imshow('MediaPipe Holistic', cv2.flip(blank, 1))

                    #if cv2.waitKey(5) & 0xFF == 27:
                    #    break
            #cv2.destroyWindow('MediaPipe Holistic')
            cap.release()
            out.release()
